In [1]:
import giskard
from typing import Sequence, Optional
from giskard.llm.client import set_default_client
from giskard.llm.client.base import LLMClient, ChatMessage
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import PromptTemplate, LLMChain

from dotenv import load_dotenv
import os
from llama_parse import LlamaParse
import nest_asyncio

import pandas as pd
from giskard.rag import KnowledgeBase
from giskard.rag import generate_testset

from langchain_community.document_loaders.text import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

nest_asyncio.apply()

/root/miniconda3/envs/cit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# mistralai/Mistral-7B-Instruct-v0.2
model_name_or_path = "NousResearch/Hermes-2-Pro-Llama-3-8B"

config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
config.max_position_embeddings = 8096
quantization_config = BitsAndBytesConfig(
llm_int8_enable_fp32_cpu_offload=True,
bnb_4bit_quant_type='nf4',
bnb_4bit_use_double_quant=True,
bnb_4bit_compute_dtype=torch.bfloat16,
load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
model_name_or_path,
config=config,
trust_remote_code=True,
quantization_config=quantization_config,
device_map="cuda",
)

# tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2', token=hf_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:26<00:00,  6.58s/it]


In [27]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
env_dir = os.path.join(parent_dir, 'app')
env_file = os.path.join(env_dir, '.env')

load_dotenv(dotenv_path=env_file)
llama_parse_key = os.getenv("LLAMA_INDEX_TOKEN")


parser = LlamaParse(
    result_type = 'markdown',
    verbose = True,
    language = 'en',
    num_workers = 8,
    api_key = llama_parse_key
)

parent_dir = os.path.dirname(current_dir)
path_to_pdf = 'app/pdf_docs'
file_path = os.path.join(parent_dir, path_to_pdf)

c = 0
documents_list = []

for doc in os.listdir(file_path):
    if doc.endswith(".pdf"):
        documents_list.append(f'{file_path}/{doc}')

    c += 1
    if c >= 5:
        break

documents = parser.load_data(documents_list)
stiched_docs = ["".join(doc.text) for doc in documents]
stiched_docs = "".join(stiched_docs)

In [53]:
# save stiched_docs as a txt file
with open('stiched_docs.txt', 'w') as f:
    f.write(stiched_docs)

In [58]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
loader = TextLoader(file_path='stiched_docs.txt')
documents = loader.load_and_split(text_splitter)

In [59]:
documents

[Document(page_content='# Noise pollution and noise complaints\n\nWhat is noise pollution?\n\n- Is considered a nuisance\n- Is dangerous to human health\n- Could damage property\n- Could damage the environment\n\nThere are no universal noise standard laws that apply in Ireland but your local authority may refer to particular noise standards. Noise pollution is outlined in the Environmental Protection Agency Act 1992.\n\nWho do I complain to about noise pollution?\n\nWhile the law does not mention an exact level or standard of noise that is illegal, if noise is affecting your quality of life, then you have a right to complain. Who you complain to depends on the source of the noise.\n\nYour first step should generally be to contact the person causing the noise to try to fix the issue. If this doesn’t work, you can take the necessary next steps to make your complaint. There are different bodies who deal with these complaints, but the 2 main places to make your complaints are:\n\n- Local a

In [74]:
df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
load_dotenv(dotenv_path=env_file)
OPENAI_API_KEY = os.getenv("OPENAI_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
MODEL = "gpt-3.5-turbo"
knowledge_base = KnowledgeBase(df)

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="Tests for Citizens Information",
)

2024-05-24 16:00:27,198 pid:2627 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


/root/miniconda3/envs/cit/lib/python3.10/site-packages/umap/umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


2024-05-24 16:00:32,374 pid:2627 MainThread giskard.rag  INFO     Found 4 topics in the knowledge base.


Generating questions: 100%|██████████| 60/60 [08:36<00:00,  8.60s/it]


In [77]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

Question 1: Who is prioritized by the National Framework of Qualifications (NFQ) service?
Reference answer: The service prioritises people who are unemployed.
Reference context:
Document 20: - Everyone over 18
- People over 16 who are not in full-time education.

The service prioritises people who are unemployed. Read more about returning to education. Further education and training qualifications

The National Framework of Qualifications (NFQ) allows the different levels of qualifications to be compared. There are 10 Levels on the NFQ. FET courses are provided at Levels 1 to 6 on the NFQ.

Progression to third-level education

If you have a qualification at NFQ Level 5 and Level 6, you can progress to third-level education with the Higher Education Links Scheme. The tertiary education programme is a new pathway to third level education. You start in a further education institution and progress to a higher education institution to complete a degree. You can also access third-level educ

In [78]:
test_set_df.to_pickle("test_set_df.pkl")

In [72]:
knowledge_base

2024-05-24 15:59:23,436 pid:2627 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


/root/miniconda3/envs/cit/lib/python3.10/site-packages/umap/umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
Exception ignored on calling ctypes callback function: <function ExecutionEngine._raw_object_cache_notify at 0x7efc83db04c0>
Traceback (most recent call last):
  File "/root/miniconda3/envs/cit/lib/python3.10/site-packages/llvmlite/binding/executionengine.py", line 178, in _raw_object_cache_notify
    def _raw_object_cache_notify(self, data):
KeyboardInterrupt: 


2024-05-24 15:59:30,865 pid:2627 MainThread giskard.rag  INFO     Found 3 topics in the knowledge base.


Generating questions:   2%|▏         | 1/60 [00:10<10:13, 10.40s/it]


KeyboardInterrupt: 

In [3]:


current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
path_to_pdf = 'app/pdf_docs'
file_path = os.path.join(parent_dir, path_to_pdf)

documents_text = []
c = 0

for doc in os.listdir(file_path):

    doc_path = os.path.join(file_path, doc)
    loader = PyPDFLoader(doc_path)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    docs = text_splitter.split_documents(pages)
    documents_text.append(docs)
    c += 1
    if c == 5:
        break

documents_text = [item for sublist in documents_text for item in sublist]

In [6]:
print(documents_text[3].page_content)

You can get a complaint form in your local district court office and will have to pay a fee
to process the complaint.
Other organisations for noise complaints
Depending on the source of the noise, you may complain elsewhere. Other places where
you can lodge a complaint include:
An Garda Síochánna
The Environmental Protection Agency (EPA)
Residential Tenancies Board (RTB)
Transport Infrastructure Ireland (TII)
You can read the gov.ie Guide to Noise Regulations  on the steps you can take if you are
experiencing a nuisance caused by noise.
How to make a noise complaint
If you plan to complain about excessive noise, try to keep a detailed list of:
The times when the noise occurred
How long the noise lasted
The levels of noise involved (if possible)
Step 1: Try to resolve the issue yourself
You should first contact the person or business causing the noise, explain that it is a
nuisance and try to solve the issue together.
Step 2: Contact your local authority or the EPA


In [7]:
pipe  = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=250
)

local_llm = HuggingFacePipeline(pipeline=pipe)

template = """can you tell me a joke about {topic}?"""
prompt = PromptTemplate(template=template, input_variables=["spaniards"])
llm_chain = LLMChain(prompt=prompt, llm=local_llm, verbose=True)

model.eval()
with torch.no_grad():
    print(llm_chain.run({"topic": "spaniards"}))


Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
can you tell me a joke about spaniards?

> Finished chain.
can you tell me a joke about spaniards? "
A Spaniard walks into a bar and orders a drink. The bartender asks, "Is there a problem?" The Spaniard replies, "No, everything's fine. I just have a little problem with my hearing. In Spain, we call this 'un poco de sordera'." The bartender laughs and says, "Well, in America, we call that a 'little white lie'." 

Here are some funny Spanish jokes:

1. Why did the Spaniard cross the road?
To get to the restaurant on the other side.

2. Why did the Spaniard break up with his girlfriend?
Because she was "un poco loca" (a little crazy).

3. What do you call a Spaniard who speaks three languages?
Un problema (a problem).

4. Why did the Spaniard go to the hospital?
To have his "hijo" (son) removed. 

5. How do you stop a Spaniard from speaking?
With a gun in his mouth.

6. What do you call a Spaniard who can't speak?
Un mudo (a mut